In [9]:
!pip3 install mysql-connector-python python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os

In [11]:
# Load environment variables from .env file
load_dotenv()

# Database connection details
db_host = "swagswipeserveur.mysql.database.azure.com"
db_user = "adminSwag"
db_password = "SwaggySwipe123"
db_database = "swagswipe"

In [12]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [13]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Query executed successfully: {query}")
    except Error as e:
        print(f"The error '{e}' occurred when executing: {query}")

In [14]:
def execute_script_from_file(connection, file_path):
    cursor = connection.cursor()
    with open(file_path, 'r') as file:
        script = file.read()

    statements = script.split(';')

    for statement in statements:
        if statement.strip():
            try:
                cursor.execute(statement)
                connection.commit()
                print(f"Statement executed successfully: {statement}")
            except Error as e:
                if e.errno == 1050:  # Error code for "Table already exists"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Table already exists, continuing...")
                elif e.errno == 1061:  # Error code for "Duplicate key name"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Duplicate key name, continuing...")
                elif e.errno == 1062:  # Error code for "Duplicate entry"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Duplicate entry, continuing...")
                elif e.errno == 1146:  # Error code for "Table doesn't exist"
                    print(f"The error '{e}' occurred when executing statement: {statement} - Table doesn't exist, continuing...")
                else:
                    print(f"The error '{e}' occurred when executing statement: {statement}")

In [15]:
def main():
    connection = create_connection(db_host, db_user, db_password, db_database)

    if connection:
        execute_script_from_file(connection, "1create.sql")
        execute_script_from_file(connection, "2contraines.sql")
        execute_script_from_file(connection, "3insert.sql")

        connection.close()

if __name__ == "__main__":
    main()

Connection to MySQL DB successful
Statement executed successfully: /* -----------------------------------------------------------
   Réinitialisation complète du schéma
-----------------------------------------------------------*/
SET FOREIGN_KEY_CHECKS = 0
Statement executed successfully: 

DROP TABLE IF EXISTS favori
Statement executed successfully: 
DROP TABLE IF EXISTS avis
Statement executed successfully: 
DROP TABLE IF EXISTS produit
Statement executed successfully: 
DROP TABLE IF EXISTS utilisateur
Statement executed successfully: 

/* ----------  utilisateur  ---------- */
CREATE TABLE utilisateur (
    id_user      INT(10) AUTO_INCREMENT PRIMARY KEY,
    nom          VARCHAR(255) NOT NULL,
    prenom       VARCHAR(255) NOT NULL,
    email        VARCHAR(255) NOT NULL UNIQUE,
    telephone    VARCHAR(12)  NOT NULL,
    mdp          VARCHAR(255) NOT NULL,
    num_civique  VARCHAR(255) NOT NULL,
    rue          VARCHAR(255) NOT NULL,
    ville        VARCHAR(255) NOT NULL,
    c